# Кибериммунный подход к разработке.<br>Шаблоны безопасности.<br>Шаблон "Обфускация чувствительных данных"

## Об авторе 

Этот блокнот разработан для вас Сергеем Соболевым, sergey.p.sobolev@kaspersky.com

Больше информации о кибериммунном подходе можно найти на странице https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82

Подписывайтесь на телеграм-канал @learning_cyberimmunity (https://t.me/learning_cyberimmunity)

Обучающие видео на тему кибериммунного подхода вы можете найти на youtube канале https://www.youtube.com/@learning_cyberimmunity/

## Ключевые технологии

- FLASK (FLux Advanced Security Kernel) - простая демонстрация FLASK в другом блокноте, здесь используем наработки https://github.com/sergey-sobolev/cyberimmune-systems-basic-demo-notebook01/blob/main/cyberimmunity-basics.ipynb, открыть на Google Colab можно по ссылке https://colab.research.google.com/github/sergey-sobolev/cyberimmune-systems-basic-demo-notebook01/blob/main/cyberimmunity-basics.ipynb
- Обфускация или шифрование данных - например, конфиденциальность которых необходимо обеспечить

Обфускация позволяет сохранить конфиденциальность данных, даже в случае компрометации внутренних компонент, до тех пор, пока данные остаются в защищённом виде.

См. так же описание шаблона https://support.kaspersky.ru/help/kce/1.0/ru-RU/info_obscurity_pattern.htm

## Сценарий примера

1. Создадим две сущности (Обработчик данных - DataProcessor и База данных - DataBase) и монитор (Monitor), который будет контролировать их взаимодействие
2. Определим политики безопасности
3. Подготовим данные, которые будут содержать некритичную и критичную части, критичную часть - имя клиента - обработчик данных должен будет обфусцировать перед отправкой в базу данных

- Каждая сущность в качестве интерфейса использует очереди сообщений, у каждой сущности есть своя «персональная» очередь, ассоциированная с ней
- DataProcessor и DataBase отправляют сообщения только в очередь Monitor
- Monitor проверяет сообщения на соответствие политикам безопасности, в случае положительного решения перенаправляет сообщение в очередь соответствующей сущности

![диаграмма](//www.plantuml.com/plantuml/png/TP71IWCn48RlUOgXFUyRY1Jn0GIFPGyXcyh2Df4aYtXJi4BHYs-o1ukBkZx3p1lvq_OGT-qfCva_t_ypMSIaGshNZS9HhxH9P6X-4wnrf2FD5FVoA1l-uNSUv8LtXABZuT2e6QdieNUhkh8krRgvJZxOwY_f4vH1dhW7OeVg6LN7N_9wXCb67lRidh__jSgyMfg3VmGFm7rm9rk4tSeR-XDHZL_VUeU4jBpKIL_eQCizQ3ov2jxO67t0-DusZRzJQZme2Z8guw2VRc0aQ5L-SLOi3mignwwv2YA0olE9Pc5TXP_x1G00?raw=true)

На этой диаграмме монитор безопасности не указывается, но подразумевается.

Очередь событий для монитора безопасности: все запросы от сущностей друг к другу должны отправляться только в неё

In [322]:
from multiprocessing import Queue
monitor_events_queue = Queue()

Зафиксируем формат сообщений

In [323]:
from dataclasses import dataclass


@dataclass
class Event:
    source: str              # отправитель
    destination: str         # получатель
    operation: str           # чего хочет (запрашиваемое действие)
    parameters: str          # с какими параметрами
    data: str or None = None # дополнительные данные

### Монитор безопасности

Ниже в методе _check_policies можно увидеть пример политики безопасности:

```python
if event.source == "DataProcessor" \
                and event.destination == "DataBase" \
                and event.operation == "read_request" \
                and event.parameters == "table_orders" \
                and event.data is None:
            authorized = True
```            

в этом примере проверяется отправитель сообщения, получатель, запрашиваемая операция и даже параметры операции. Это довольно жёсткий вариант, очевидно, количество проверок можно уменьшить.

А пока это место для экспериментов, как можно из монитора безопасности заблокировать взаимодействие между сущностями.

Используем стандартные пакеты для организации многопоточной работы примера

In [324]:
from multiprocessing import Queue, Process
from multiprocessing.queues import Empty
from time import sleep

Формат управляющих команд для монитора или других сущностей при необходимости

In [325]:
@dataclass
class ControlEvent:
    operation: str

Класс, реализующий поведение монитора безопасности

In [326]:
class Monitor(Process):

    def __init__(self, events_q: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        self._events_q = events_q  # очередь событий для монитора (входящие сообщения)
        self._control_q = Queue()  # очередь управляющих команд (например, для остановки монитора)
        self._entity_queues = {}   # словарь очередей известных монитору сущностей
        self._force_quit = False   # флаг завершения работы монитора

    # регистрация очереди новой сущности
    def add_entity_queue(self, entity_id: str, queue: Queue):
        print(f"[{self.__class__.__name__}] регистрируем сущность {entity_id}")
        self._entity_queues[entity_id] = queue

    # проверка политик безопасности        
    def _check_policies(self, event):
        print(f'[{self.__class__.__name__}] обрабатываем событие {event}')

        # default deny: всё, что не разрешено, запрещено по умолчанию!
        authorized = False

        # проверка на входе, что это экземпляр класса Event, 
        # т.е. имеет ожидаемый формат
        if not isinstance(event, Event):
            return False

        # 
        #  политики безопасности
        #

        # запрос на чтение данных из базы для обработки
        if event.source == "DataProcessor" \
                and event.destination == "DataBase" \
                and event.operation == "read_request" \
                and event.parameters == "table_orders" \
                and event.data is None:
            authorized = True

        # ответ на запрос на чтение данных из базы для обработки
        # event.data не проверяем, т.к. может быть пустым или нет в 
        # зависимости от наличия данных
        if event.source == "DataBase" \
                and event.destination == "DataProcessor" \
                and event.operation == "read_response" \
                and event.parameters == "table_orders":
            authorized = True            

        # запрос на запись обработанных данных в базу 
        if event.source == "DataProcessor" \
                and event.destination == "DataBase" \
                and event.operation == "write_request" \
                and event.parameters == "table_orders" \
                and event.data is not None:
            authorized = True

        if authorized is False:
            print(f"[{self.__class__.__name__}] событие не разрешено политиками безопасности")
        return authorized

    # выполнение разрешённого запроса
    # метод должен вызываться только после проверки политик безопасности
    def _proceed(self, event):
        print(f'[{self.__class__.__name__}] отправляем запрос {event}')
        try:
            # найдём очередь получателя события
            dst_q: Queue = self._entity_queues[event.destination]
            # и положим запрос в эту очередь
            dst_q.put(event)
        except  Exception as e:
            # например, запрос пришёл от или для неизвестной сущности
            print(f"[{self.__class__.__name__}] ошибка выполнение запроса {e}")

    # основной код работы монитора безопасности    
    def run(self):
        print(f'[{self.__class__.__name__}] старт')

        # в цикле проверяет наличие новых событий, 
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                # ожидание сделано неблокирующим, 
                # чтобы можно было завершить работу монитора, 
                # не дожидаясь нового сообщения
                event = self._events_q.get_nowait()
                # сюда попадаем только в случае получение события,
                # теперь нужно проверить политики безопасности
                authorized = self._check_policies(event)
                if authorized:
                    # если политиками запрос авторизован - выполняем
                    self._proceed(event)
            except Empty:
                # сюда попадаем, если новых сообщений ещё нет,
                # в таком случае немного подождём
                sleep(0.5)
            except Exception as e:
                # что-то пошло не так, выведем сообщение об ошибке
                print(f"[{self.__class__.__name__}] ошибка обработки {e}, {event}")
            self._check_control_q()
        print(f'[{self.__class__.__name__}] завершение работы')

    # запрос на остановку работы монитора безопасности для завершения работы
    # может вызываться вне процесса монитора
    def stop(self):
        # поскольку монитор работает в отдельном процессе,
        # запрос помещается в очередь, которая проверяется из процесса монитора
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность DataProcessor

Взаимодействует с базой данных для обработки данных и записи результатов обработки.

In [334]:
class DataProcessor(Process):

    def __init__(self, monitor_queue: Queue, mode='write'):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        # очередь управляющих команд (например, для остановки)
        self._control_q = Queue()
        self._force_quit = False
        self._mode = mode

    # выдаёт собственную очередь для взаимодействия
    def entity_queue(self):
        return self._own_queue

    #тестовые данные
    @staticmethod
    def _sample_data():
        return {
            "id": "12345",
            "client": "John Doe",    # персональные данные! это поле должно быть обфусцировано при передаче в недоверенные сущности       
            "order_details": {
                "name": "cisplatin",
                "quantity": 12,
                "payment_id": "aaa-bbb-cccc",
                "total": 10000
            }
        }

    @staticmethod
    def encrypt_field(src, field_name) -> dict:
        # исключительно для демонстрационных целей, чтобы не усложнять,
        # используем собственную реализацию простейшего перестановочного шифра
        # этот код нельзя использовать в реальных системах ни при каких условиях!
        key = 3
        plain_text = src[field_name].upper()
        cipher = ""
        for char in plain_text:
            if char not in ' ,.':
                cipher +=  chr(ord('A') + (ord(char) - ord('A') + key) % 26)
            else:
                cipher += char
        src[field_name] = cipher
        return src


    @staticmethod
    def decrypt_field(src, field_name) -> dict:
        # расшифровка данных
        key = 3
        cipher = src[field_name].upper()
        plain_text = ""
        for char in cipher:
            if char not in ' ,.':
                plain_text +=  chr(ord('A') + (ord(char) - ord('A') + 26 - key) % 26)
            else:
                plain_text += char
        src[field_name] = plain_text.title()
        return src

    def send_data(self):
        # отправка тестовых данных
        print(f'[{self.__class__.__name__}] отправляем тестовый запрос на запись данных')

        # запишем тестовые данные
        data = self.encrypt_field(self._sample_data(), "client")
        print(f'[{self.__class__.__name__}] записываем данные {data}')
        event = Event(source=self.__class__.__name__,
                    destination='DataBase',
                    operation='write_request',
                    parameters='table_orders',
                    data=data
                    )
        self.monitor_queue.put(event)

    def read_data(self):
        print(f'[{self.__class__.__name__}] отправляем тестовый запрос на чтение данных')
        event = Event(source=self.__class__.__name__,
                    destination='DataBase',
                    operation='read_request',
                    parameters='table_orders',
                    data=None
                    )
        self.monitor_queue.put(event)

    # основной рабочий цикл сущности (приём и отправка данных)
    def run(self):
        print(f'[{self.__class__.__name__}] старт')
           
        while self._force_quit is False:
            try:
                event: Event = self._own_queue.get_nowait()
                if event.operation == "read_response":                                            
                    print(f"[{self.__class__.__name__}] {event.source} получили данные {event.parameters}, {event.data}")
                    client_name = self.decrypt_field(event.data, 'client')['client']
                    print(f"[{self.__class__.__name__}] имя клиента: {client_name}")
                
            except Empty:                
                sleep(0.2)
                self._check_control_q()
        
        print(f'[{self.__class__.__name__}] завершение работы')

    # метод, который нужно вызвать для остановки работы сущности
    def stop(self):
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass        


### Сущность DataBase

Эта сущность предоставляет интерфейс для работы с данными, абстрагируя от уровня записи в файловую систему.

In [328]:
class DataBase(Process):

    def __init__(self, monitor_queue: Queue):
        # конструктор аналогичный
        super().__init__()
        self.monitor_queue = monitor_queue
        self._own_queue = Queue()
        self._control_q = Queue()
        self._force_quit = False
        self._data = {}

    def entity_queue(self):
        return self._own_queue

    def send_data(self, details: Event):
        try:
            data = self._data[details.parameters]
            response = Event(
                source = self.__class__.__name__,
                destination= details.source,
                operation='read_response',
                parameters=details.parameters,
                data=data
            )
            self.monitor_queue.put(response)
        except Exception as e:
            print(f'{self.__class__.__name__} ошибка обработки запроса на чтение: {e}')

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')
        while self._force_quit is False:
            try:
                event: Event = self._own_queue.get_nowait()
                if event.operation == "write_request":
                    print(f"[{self.__class__.__name__}] {event.source} захотел записать данные {event.parameters}")
                    self._data[event.parameters] = event.data
                elif event.operation == "read_request":
                    print(f"[{self.__class__.__name__}] {event.source} захотел прочитать данные {event.parameters}")
                    self.send_data(event)
            except Empty:                
                sleep(0.2)
                self._check_control_q()
        print(f'[{self.__class__.__name__}] завершение работы')

    def stop(self):
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[{self.__class__.__name__}] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Инициализируем монитор и сущности

In [329]:
monitor = Monitor(monitor_events_queue)
data_processor = DataProcessor(monitor_events_queue)
data_base = DataBase(monitor_events_queue)

регистрируем очереди сущностей в мониторе

In [330]:
monitor.add_entity_queue(data_processor.__class__.__name__, data_processor.entity_queue())
monitor.add_entity_queue(data_base.__class__.__name__, data_base.entity_queue())

[Monitor] регистрируем сущность DataProcessor
[Monitor] регистрируем сущность DataBase


### Запускаем всё

Ожидаемая последовательность событий

![Диаграмма последовательности вызовов](//www.plantuml.com/plantuml/png/rLGzhjf04Ext55k-9yG5KaGeIXif1qYpSLnO2ZQo5sfYo8yK99f8IPighR6mx40mLvYvKMPiLimY495IlGPcT-Vd-sQ-yIYp9hNJyNireVyeY2OcjlgbiUPrcWHXbYMfz_8gYIChxfpt2vE5IfcfJUBf-6sOAdNorCFdBkYPnW-OGu5pp9KBudTd-muI0tl92pzNdDXGco1Diap13pX0Wv-WWGBsz5j3mITSQpQXX8fYla15ReKAutUQevMVupiv7g6X4YiDMpeSu81VyQFmlmSFMjZWd48tq1BG7HJwWSeskAJR5iekQGFLewQtbj8lwRHbM0uxhlgYRs89GSEH6P3ZLdUPZtHWMfpbjuPImuREDIvyY68EWm71gnr5A33tQ4B5MTnChiwnAzhslMHg3AhbRXEU1dOonkKP3dbat1LYmLvL1_z2QYQmqSpOy5pU6Nwkg_sLwN5DRQjlZR6VTZ8BqtCX-2OQ8Dl3LxU4yejgLVqW_rKeSWfzst21dw6vCWKIxzEVWnAX5-mvWB7dp7kFQn_4_p6-t4Z2AWkoUIOdw8gGEpj-W_q3Rx7RTFVjUVJL8UKd2yBFqpApeagoAlWZjIYreLNVYruxuabgIEXmH2Pzv_y2)

In [331]:
monitor.start()
data_base.start()
data_processor.start()
data_processor.send_data()
sleep(1)
data_processor.read_data()
sleep(2)

[Monitor] старт
[DataBase] старт
[DataProcessor] старт
[DataProcessor] отправляем тестовый запрос на запись данных
[DataProcessor] записываем данные {'id': '12345', 'client': 'MRKQ GRH', 'order_details': {'name': 'cisplatin', 'quantity': 12, 'payment_id': 'aaa-bbb-cccc', 'total': 10000}}
[Monitor] обрабатываем событие Event(source='DataProcessor', destination='DataBase', operation='write_request', parameters='table_orders', data={'id': '12345', 'client': 'MRKQ GRH', 'order_details': {'name': 'cisplatin', 'quantity': 12, 'payment_id': 'aaa-bbb-cccc', 'total': 10000}})
[Monitor] отправляем запрос Event(source='DataProcessor', destination='DataBase', operation='write_request', parameters='table_orders', data={'id': '12345', 'client': 'MRKQ GRH', 'order_details': {'name': 'cisplatin', 'quantity': 12, 'payment_id': 'aaa-bbb-cccc', 'total': 10000}})
[DataBase] DataProcessor захотел записать данные table_orders
[Monitor] обрабатываем событие Event(source='DataProcessor', destination='DataBase

### Завершение работы процессов

Сначала отправим запросы на останов процессов

In [332]:
monitor.stop()
data_processor.stop()
data_base.stop()


Теперь подождём завершения их работы

In [333]:
data_processor.join()
data_base.join()
monitor.join()

[DataBase] проверяем запрос ControlEvent(operation='stop')
[DataProcessor] проверяем запрос ControlEvent(operation='stop')[DataBase] завершение работы

[DataProcessor] завершение работы
[Monitor] проверяем запрос ControlEvent(operation='stop')
[Monitor] завершение работы


## Заключение

В этом блокноте продемонстрирован шаблон обфускации чувствительных данных в сочетании с подходом FLASK, заключающемся в тотальном контроле взаимодействия сущностей с помощью монитора безопасности и простейших политик.

Благодаря обфускации, мы можем не беспокоиться об утечке данных из недоверенных сущностей, при условии, что вместе с данными не утекает и механизм восстановления оригинала (который должен быть доступен только доверенным компонентам). 

В некоторых случаях обфускация может быть односторонней, т.е. не предполагать восстановление исходной версии (например, случайная перестановка символов или замена на другие) - это может быть полезно для полноценной работы большинства подсистем, которым важно работать с полной базой, но необязательно с аутентичными данными - это, например, различная аналитика, машинное обучение или тестирование.

Очевидно, оригинальные данные должны быть доступны только доверенным сущностм, а обфусцированные - вообще всем, кому это нужно с точки зрения выполнения бизнес-функций, разработки или наладки системы.

## Упражнения

### Уровень "Новичок"

- в передаваемом сообщении добавьте конфиденциальное поле "адрес доставки" (delivery_address) и защитите его по аналогии с именем клиента

### Уровень "Продвинутый"

- защитите также сумму заказа (order_details -> total)

### Уровень "Спец"

_Возможно, для задач этого уровня сложности имеет смысл перенести разработку кода в VS Code или PyCharm._

- вместо простого "самодельного" шифра используйте алгоритм симметричного шифрования для защиты и контроля целостнсти конфиденциальных данных.
- добавьте ещё одну недоверенную сущность FileSystem, которая должна будет реализовывать запись данных в файл
  - только DataBase может взаимодействовать с FileSystem; интерфейс - read/write. При записи данных в файл - если файла нет - создать; при чтении - если файла нет - вернуть None в поле data

### Уровень "Крутой"

_Возможно, для задач этого уровня сложности имеет смысл перенести разработку кода в VS Code или PyCharm._

- добавьте ещё одну доверенную сущность DeliveryProcessor, которая должна также иметь доступ к конфиденциальной информации о заказах для осуществления процесса доставки
  - DeliveryProcessor будет взаимодействовать с DataBase
  - DataProcessor должен использовать асимметричное шифрование для защиты конфиденциальных данных
  - DeliveryProcessor должен контролировать целостность конфиденциальных данных, в реализации DeliveryProcessor публичный ключ DataProcessor может быть реализован как параметр (константа)
- В мониторе безопасности реализуйте контроль подписи конфиденциальных данных как политику безопасности:
  - при чтении данных из базы необходимо проверить, что конфиденциальная информация правильно зашифрована